# Helpdesk Hard

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app12-3")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@7dac6a87

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@343aa440

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val shift = hiveCxt.table("sqlzoo.Shift")
val staff = hiveCxt.table("sqlzoo.Staff")
val issue = hiveCxt.table("sqlzoo.Issue")
val shift_type = hiveCxt.table("sqlzoo.Shift_type")
val level = hiveCxt.table("sqlzoo.Level")
val customer = hiveCxt.table("sqlzoo.Customer")
val caller = hiveCxt.table("sqlzoo.Caller")

shift: DataFrame = [shift_date: string, shift_type: string ... 4 more fields]
staff: DataFrame = [staff_code: string, first_name: string ... 2 more fields]
issue: DataFrame = [call_date: string, call_ref: int ... 5 more fields]
shift_type: DataFrame = [shift_type: string, start_time: string ... 1 more field]
level: DataFrame = [level_code: int, manager: string ... 2 more fields]
customer: DataFrame = [company_ref: int, company_name: string ... 6 more fields]
caller: DataFrame = [caller_id: int, company_ref: int ... 2 more fields]

## 11.
Show the manager and number of calls received for each hour of the day on 2017-08-12

```
+---------+---------------+----+
| Manager | Hr            | cc |
+---------+---------------+----+
| LB1     | 2017-08-12 08 |  6 |
| LB1     | 2017-08-12 09 | 16 |
| LB1     | 2017-08-12 10 | 11 |
| LB1     | 2017-08-12 11 |  6 |
| LB1     | 2017-08-12 12 |  8 |
| LB1     | 2017-08-12 13 |  4 |
| AE1     | 2017-08-12 14 | 12 |
| AE1     | 2017-08-12 15 |  8 |
| AE1     | 2017-08-12 16 |  8 |
| AE1     | 2017-08-12 17 |  7 |
| AE1     | 2017-08-12 19 |  5 |
+---------+---------------+----+
```

In [5]:
(issue.withColumn("date_call", to_date(issue("Call_date")))
 .withColumn("hr", date_format(issue("Call_date"), "yyyy-MM-dd HH"))
 .filter(col("date_call")==="2017-08-12")
 .join(shift, (col("Taken_by")===col("Operator")) && 
       (col("date_call")===col("Shift_date")))
 .groupBy("Manager", "hr")
 .agg(count("Call_ref").alias("cc"))
 .orderBy("hr")
 .showHTML())

Manager,hr,cc
LB1,2017-08-12 08,6
LB1,2017-08-12 09,16
LB1,2017-08-12 10,11
LB1,2017-08-12 11,6
LB1,2017-08-12 12,8
LB1,2017-08-12 13,4
AE1,2017-08-12 14,12
AE1,2017-08-12 15,8
AE1,2017-08-12 16,8
AE1,2017-08-12 17,7


## 12.
**80/20 rule. It is said that 80% of the calls are generated by 20% of the callers. Is this true? What percentage of calls are generated by the most active 20% of callers.**

Note - Andrew has not managed to do this in one query - but he believes it is possible.

```
+---------+
| t20pc   |
+---------+
| 32.2581 |
+---------+
```

In [6]:
(issue.groupBy("Caller_id")
 .agg(count("Caller_id").alias("n"))
 .orderBy(col("n").desc)
 .limit((issue.select("Caller_id").distinct().count() * 0.2).floor.toInt)
 .agg(sum("n").alias("sum_n"))
 .withColumn("t20pc", round(lit(100) * col("sum_n") / issue.count(), 4))
 .select("t20pc")
 .showHTML())

t20pc
32.2581


## 13.
**Annoying customers. Customers who call in the last five minutes of a shift are annoying. Find the most active customer who has never been annoying.**

```
+--------------+------+
| Company_name | abna |
+--------------+------+
| High and Co. |   20 |
+--------------+------+
```

In [7]:
val annoy = (issue.withColumn("date_call", to_date(issue("Call_date")))
         .join(shift, (col("Taken_by")===col("Operator")) && 
                      (col("date_call")===col("Shift_date")))
         .join(shift_type, "Shift_type")
         .join(caller, "Caller_id")
         .join(customer, "Company_ref")
         .select("Company_ref", "Shift_date", "End_time", "Call_date")
         .withColumn("Shift_end", concat_ws(
             " ", col("Shift_date").cast("string"), col("End_time")))
         .withColumn("till_shiftend", 
                     (to_timestamp(col("Shift_end")) - to_timestamp(col("Call_date")))
                     .cast("long"))
         .filter(col("till_shiftend") <= 5*60))

(issue.join(caller, "Caller_id")
 .join(customer, "Company_ref")
 .join(annoy, "Company_ref", joinType="left_anti")
 .groupBy("Company_ref", "Company_name")
 .agg(count("Caller_id").alias("abna"))
 .orderBy(col("abna").desc)
 .limit(1)
 .showHTML())

Company_ref,Company_name,abna
146,High and Co.,20


annoy: Dataset[Row] = [Company_ref: int, Shift_date: string ... 4 more fields]

## 14.
**Maximal usage. If every caller registered with a customer makes a call in one day then that customer has "maximal usage" of the service. List the maximal customers for 2017-08-13.**

```
+-------------------+--------------+-------------+
| company_name      | caller_count | issue_count |
+-------------------+--------------+-------------+
| Askew Inc.        |            2 |           2 |
| Bai Services      |            2 |           2 |
| Dasher Services   |            3 |           3 |
| High and Co.      |            5 |           5 |
| Lady Retail       |            4 |           4 |
| Packman Shipping  |            3 |           3 |
| Pitiable Shipping |            2 |           2 |
| Whale Shipping    |            2 |           2 |
+-------------------+--------------+-------------+
```

In [8]:
val iss = (issue.filter((col("Call_date")>="2017-08-13") &&
                    (col("Call_date")<"2017-08-14"))
       .join(caller, "Caller_id", joinType="right")
       .join(customer, "Company_ref", joinType="left")
       .groupBy("Caller_id", "Company_ref", "Company_name")
       .agg(count("Call_ref").alias("n"))
       .withColumn("iss", (col("n")>0).cast("integer")))

(iss.groupBy("Company_name")
 .agg(count("Company_ref").alias("caller_count"),
      sum("iss").alias("issue_count"))
 .filter(col("caller_count")===col("issue_count"))
 .orderBy("Company_name")
 .showHTML())

Company_name,caller_count,issue_count
Askew Inc.,2,2
Bai Services,2,2
Dasher Services,3,3
High and Co.,5,5
Lady Retail,4,4
Packman Shipping,3,3
Pitiable Shipping,2,2
Whale Shipping,2,2


iss: DataFrame = [Caller_id: int, Company_ref: int ... 3 more fields]

## 15.
**Consecutive calls occur when an operator deals with two callers within 10 minutes. Find the longest sequence of consecutive calls – give the name of the operator and the first and last call date in the sequence.**

```
+----------+---------------------+---------------------+-------+
| taken_by | first_call          | last_call           | calls |
+----------+---------------------+---------------------+-------+
| AB1      | 2017-08-14 09:06:00 | 2017-08-14 10:17:00 |    24 |
+----------+---------------------+---------------------+-------+
```

In [9]:
(issue.withColumn("lag", lag("Call_date", 1).over(
    Window.partitionBy("Taken_by").orderBy("Taken_by", "Call_date")))
 .withColumn("flag", ((to_timestamp(col("Call_date")) - 
                       to_timestamp(col("lag"))).cast("long") > 10*60).cast("integer"))
 .withColumn("grp", sum("flag").over(
     Window.partitionBy("Taken_by").orderBy("Taken_by", "Call_date")))
 .groupBy("Taken_by", "grp")
 .agg(min("Call_date").alias("first_call"),
      max("Call_date").alias("last_call"),
      count("Caller_id").alias("n_calls"))
 .orderBy(col("n_calls").desc)
 .select("Taken_by", "first_call", "last_call", "n_calls")
 .limit(1)
 .showHTML())

Taken_by,first_call,last_call,n_calls
AB1,2017-08-14 09:06:00.0,2017-08-14 10:17:00.0,24


In [10]:
spark.stop()